# Conversation Graphs

This notebook creates edge and node lists from the Twitter and Reddit sample conversations for use in visualizations. This relies on having the s3://tweets.pull/ s3 bucket mounted in a data directory above the directory where this notebook is.

In [9]:
import json
import pathlib

tweets_dir = pathlib.Path('../data/tweets.pull')
reddit_dir = pathlib.Path('../data/reddit.pull')
convs_dir = pathlib.Path('./convs')

## Get the Data

We want to iterate through the files in the sample zip files and get the CSVs as Pandas DataFrames. This function when given a location where the sample zip files live, will iterate through the zips and look for CSVs in them which it will return along with the name of the dataset (matches the search criteria used to generate the data) and the conversation id (a tweet id or Reddit post id).

In [2]:
import io
import csv
import pandas
import zipfile

def get_conv_df(data_dir):
    for zip_path in data_dir.glob('*_30.zip'):
        z = zipfile.ZipFile(zip_path)
        for filename in z.namelist():
            if filename.endswith('.csv'):
                name, conv_id = filename.strip('.csv').split('/')
                table = pandas.read_csv(z.open(filename, 'r'))
                yield name, conv_id, table

We can test it out to get the first tweet dataset:

In [3]:
twitter_name, twitter_conv_id, twitter_df = next(get_conv_df(tweets_dir))
print(f'dataset name: {twitter_name}')
print(f'conversation_id: {twitter_conv_id}')

twitter_df

dataset name: tweets_wealth_convs_30
conversation_id: 1277729142390304768


,id,created_at,text,attachments.media,attachments.media_keys,attachments.poll.duration_minutes,attachments.poll.end_datetime,attachments.poll.id,attachments.poll.options,attachments.poll.voting_status,...,source,withheld.scope,withheld.copyright,withheld.country_codes,type,__twarc.retrieved_at,__twarc.url,__twarc.version,Unnamed: 93,sentiment
0,1277729145917771776,2020-06-29T22:22:22.000Z,-Minimum $1000 cash payments to all Americans ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Twitter Web App,NaN,NaN,NaN,replied_to,2021-09-05T13:12:11+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.4.3,NaN,0.7269
1,1277729148509851648,2020-06-29T22:22:22.000Z,#RussForUs #NJ06\n\n#AndrewYangEndorsed\n\nCan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Twitter Web App,NaN,NaN,NaN,NaN,2021-09-05T13:12:11+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.4.3,NaN,0.0000
2,1277729142390304768,2020-06-29T22:22:21.000Z,A Universal Basic Income will unleash our pote...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Twitter Web App,NaN,NaN,NaN,replied_to,2021-09-05T13:12:11+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.4.3,NaN,-0.6238


And we can try it on the Reddit data:

In [10]:
reddit_name, reddit_conv_id, reddit_df = next(get_conv_df(reddit_dir))
print(f'dataset name: {reddit_name}')
print(f'conversation_id: {reddit_conv_id}')

reddit_df

dataset name: reddit_racial_wealth_gap_convs_30
conversation_id: m4ljb0


,all_awardings,approved_at_utc,associated_award,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,...,retrieved_on,score,send_replies,stickied,subreddit,subreddit_id,top_awarded_type,total_awards_received,treatment_tags,sentiment
0,[],NaN,NaN,oath2order,NaN,NaN,[],NaN,NaN,NaN,...,1615701284,1,True,False,scotus,t5_2rfsw,NaN,0,[],0.0000
1,[],NaN,NaN,King_Posner,NaN,NaN,[],NaN,NaN,NaN,...,1615721998,1,True,False,scotus,t5_2rfsw,NaN,0,[],0.7271
2,[],NaN,NaN,Sandra_Day_Rehnquist,NaN,NaN,[],NaN,NaN,NaN,...,1615725644,1,True,False,scotus,t5_2rfsw,NaN,0,[],0.5423
3,[],NaN,NaN,King_Posner,NaN,NaN,[],NaN,NaN,NaN,...,1615747166,1,True,False,scotus,t5_2rfsw,NaN,0,[],0.0307
4,[],NaN,NaN,arbivark,NaN,NaN,[],NaN,NaN,NaN,...,1615747249,1,True,False,scotus,t5_2rfsw,NaN,0,[],0.1779
5,[],NaN,NaN,Cwagmire,NaN,NaN,[],NaN,NaN,NaN,...,1615749793,1,True,False,scotus,t5_2rfsw,NaN,0,[],-0.7486
6,[],NaN,NaN,Cwagmire,NaN,NaN,[],NaN,NaN,NaN,...,1615750133,1,True,False,scotus,t5_2rfsw,NaN,0,[],0.4266
7,[],NaN,NaN,[deleted],NaN,NaN,NaN,NaN,NaN,dark,...,1615750257,1,True,False,scotus,t5_2rfsw,NaN,0,[],0.0000
8,[],NaN,NaN,King_Posner,NaN,NaN,[],NaN,NaN,NaN,...,1615750776,1,True,False,scotus,t5_2rfsw,NaN,0,[],0.8805
9,[],NaN,NaN,Sandra_Day_Rehnquist,NaN,NaN,[],NaN,NaN,NaN,...,1615751097,1,True,False,scotus,t5_2rfsw,NaN,0,[],0.7448


## Extract Network

The Twitter and Reddit data is shaped differently but we can process each one into a datastructure of nodes and edges. It's actually kind of tedious because the head node in both Twitter and Reddit was not included in the conversation thread other than as an id that is being pointed at. So this code goes through some contortions to get it after the fact, to make it easier to display the network.

In [54]:
import os
import time

import twarc
twitter = twarc.client2.Twarc2(bearer_token=os.environ.get('BEARER_TOKEN'))

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()

def make_twitter_graph(df):   
    nodes = []
    edges = []
    
    for row in df.iloc():
        
        node_id = str(row['id'])
        if node_id not in [n['id'] for n in nodes]:
            nodes.append({
                "id": node_id,
                "url": f"https://twitter.com/{row['author.username']}/status/{row['id']}",
                "user": row["author.username"],
                "text": row["text"],
                "sentiment": row["sentiment"]
            })
        
        if type(row['referenced_tweets']) == str:
            reply_to_id = None
            for ref in json.loads(row['referenced_tweets']):
                if ref['type'] == 'replied_to':
                    reply_to_id = ref['id']
            if reply_to_id:
                edges.append((node_id, reply_to_id))
                
    # make sure all the edge ids have node information
    # the head of the conversation thread is usually missing
    
    node_ids = set([n['id'] for n in nodes])
    all_node_ids = set([n[0] for n in edges] + [n[1] for n in edges])
    missing_ids = all_node_ids - node_ids
    
    for result in twitter.tweet_lookup(list(missing_ids)):
        if 'data' not in result:
            continue
        for tweet in twarc.ensure_flattened(result):
            nodes.append({
                "id": tweet["id"],
                "url": f"https://twitter.com/{tweet['author']['username']}/status/{tweet['id']}",
                "user": tweet['author']['username'],
                "text": tweet['text'],
                "sentiment": vader.polarity_scores(tweet['text'])['compound']
            })  
        time.sleep(1)
        
    return {"nodes": nodes, "edges": edges}
        
make_twitter_graph(twitter_df)

{'nodes': [{'id': '1277729145917771776',
   'url': 'https://twitter.com/UBI2021/status/1277729145917771776',
   'user': 'UBI2021',
   'text': '-Minimum $1000 cash payments to all Americans + $500 per dependent\\n-Eradicate poverty\\n-Narrow the racial wealth gap \\n-End the “scarcity mindset” transition to a “plentiful mindset” \\n-Incentivize socially beneficial work or education to curb the ramifications of automation\\n\\n #NJ06',
   'sentiment': 0.7269},
  {'id': '1277729148509851648',
   'url': 'https://twitter.com/UBI2021/status/1277729148509851648',
   'user': 'UBI2021',
   'text': '#RussForUs #NJ06\\n\\n#AndrewYangEndorsed\\n\\nCan you contribute today?\\nhttps://t.co/tyAKzgNT1j',
   'sentiment': 0.0},
  {'id': '1277729142390304768',
   'url': 'https://twitter.com/UBI2021/status/1277729142390304768',
   'user': 'UBI2021',
   'text': 'A Universal Basic Income will unleash our potential as human beings.\\n\\nThe incumbent Corporate Democrat Frank Pallone, DOES NOT support this an

And we can make a similar function for the Reddit conversations:

In [62]:
import requests

def make_reddit_graph(df):   
    nodes = []
    edges = []
    
    for row in df.iloc():
        
        node_id = row['id']
        if node_id not in [n['id'] for n in nodes]:
            nodes.append({
                "id": row["id"],
                "url": "https://www.reddit.com" + row["permalink"],
                "user": row["author"],
                "text": row["body"],
                "sentiment": row["sentiment"]
            })
        
        if row['parent_id']:
            edges.append((node_id, row['parent_id'].split('_')[1]))
            
    # make sure all the edge ids have node information
    # the head of the conversation thread is usually missing
    
    node_ids = set([n['id'] for n in nodes])
    all_node_ids = set([n[0] for n in edges] + [n[1] for n in edges])
    missing_ids = all_node_ids - node_ids
    
    time.sleep(1)
    resp = requests.get(f'https://api.pushshift.io/reddit/search/submission/?ids={",".join(list(missing_ids))}')
    if resp.status_code == 200:
        for post in resp.json().get('data', []):
            nodes.append({
                "id": post["id"],
                "url": post['full_link'],
                "user": post['author'],
                "text": post.get('selftext', ''),
                "sentiment": vader.polarity_scores(post.get('selftext', ''))['compound']
            })  
    
    return {"nodes": nodes, "edges": edges}
        
make_reddit_graph(reddit_df)

{'nodes': [{'id': 'gqvihaf',
   'url': 'https://www.reddit.com/r/scotus/comments/m4ljb0/the_racial_wealth_gap_is_a_civil_liberties_issue/gqvihaf/',
   'user': 'oath2order',
   'text': "What's the relevance to this sub?",
   'sentiment': 0.0},
  {'id': 'gqw3iva',
   'url': 'https://www.reddit.com/r/scotus/comments/m4ljb0/the_racial_wealth_gap_is_a_civil_liberties_issue/gqw3iva/',
   'user': 'King_Posner',
   'text': 'This one has it, if you read. The constitutional law evolutionary argument is pretty solid, I don’t agree with the conclusion as they only prove economic class is an issue, not an intersected, legally speaking, but it’s a good analysis of the caselaw.',
   'sentiment': 0.7271},
  {'id': 'gqw7kg4',
   'url': 'https://www.reddit.com/r/scotus/comments/m4ljb0/the_racial_wealth_gap_is_a_civil_liberties_issue/gqw7kg4/',
   'user': 'Sandra_Day_Rehnquist',
   'text': 'What do they want the court to do about it, rule that wealth must be redistributed on the basis of race?',
   'sent

## Save the Data

We're going to just save off the node and edge data to some files and do the visualization somewhere else.

In [58]:
for name, conv_id, df in get_conv_df(tweets_dir):
    g = make_twitter_graph(df)
    f = convs_dir / f"{name}_{conv_id}.json"
    json.dump(g, f.open('w'), indent=2)
    
    edges_csv = convs_dir / f"{name}_{conv_id}_edges.csv"
    out = csv.writer(edges_csv.open('w'))
    out.writerow(['source', 'target'])
    for edge in g['edges']:
        out.writerow(edge)
    
    nodes_csv = convs_dir / f"{name}_{conv_id}_nodes.csv"
    out = csv.DictWriter(nodes_csv.open('w'), fieldnames=['id', 'url', 'user', 'text', 'sentiment'])
    for node in g['nodes']:
        out.writerow(node)
    
    print(f)

convs/tweets_wealth_convs_30_1277729142390304768.json
convs/tweets_wealth_convs_30_1300299144435773441.json
convs/tweets_wealth_convs_30_1269772851416051713.json
convs/tweets_wealth_convs_30_1394330467474698245.json
convs/tweets_wealth_convs_30_1281424682953121794.json
convs/tweets_wealth_convs_30_1314248442936528898.json
convs/tweets_wealth_convs_30_1325261109516001280.json
convs/tweets_wealth_convs_30_1351464750559985664.json
convs/tweets_wealth_convs_30_1352109589169274880.json
convs/tweets_wealth_convs_30_1271562825056440321.json
convs/tweets_wealth_convs_30_1320859153590480899.json
convs/tweets_wealth_convs_30_1303108284216258561.json
convs/tweets_wealth_convs_30_1272872149548662784.json
convs/tweets_wealth_convs_30_1352096623237738497.json
convs/tweets_wealth_convs_30_1395157556876742656.json
convs/tweets_wealth_convs_30_1298894279365087233.json
convs/tweets_wealth_convs_30_1269731880183742470.json
convs/tweets_wealth_convs_30_1363628008179961862.json
convs/tweets_wealth_convs_30

/tmp/ipykernel_185726/3140538378.py:1: DtypeWarning: Columns (6,8,9,10,38,58) have mixed types.Specify dtype option on import or set low_memory=False.
  for name, conv_id, df in get_conv_df(tweets_dir):


convs/tweets_black_us_convs_30_1367879818633691144.json
convs/tweets_black_us_convs_30_1365463908115484674.json
convs/tweets_black_us_convs_30_1389770291413340160.json
convs/tweets_black_us_convs_30_1360669119897600001.json
convs/tweets_black_us_convs_30_1374469084117528577.json
convs/tweets_black_us_convs_30_1378775345755791360.json
convs/tweets_black_us_convs_30_1359139334952263680.json
convs/tweets_black_us_convs_30_1383854084231290881.json
convs/tweets_black_us_convs_30_1359575781534404608.json
convs/tweets_black_us_convs_30_1376135743416983557.json
convs/tweets_black_us_convs_30_1369674132573585410.json
convs/tweets_black_us_convs_30_1368209387798605825.json
convs/tweets_black_us_convs_30_1370139291704233990.json
convs/tweets_black_us_convs_30_1358177449817890818.json
convs/tweets_black_us_convs_30_1376928803134509056.json
convs/tweets_black_us_convs_30_1374411795473952775.json
convs/tweets_black_us_convs_30_1395025446362832901.json
convs/tweets_black_us_convs_30_13674677394183987

/tmp/ipykernel_185726/3140538378.py:1: DtypeWarning: Columns (6,8,9,42,43,58,86,87,88) have mixed types.Specify dtype option on import or set low_memory=False.
  for name, conv_id, df in get_conv_df(tweets_dir):


convs/tweets_business_convs_30_1274554545100730368.json
convs/tweets_business_convs_30_1291088469461278720.json
convs/tweets_business_convs_30_1375896975263080453.json
convs/tweets_business_convs_30_1385292701118128136.json
convs/tweets_business_convs_30_1288850928632832000.json
convs/tweets_business_convs_30_1271958325014470660.json
convs/tweets_business_convs_30_1372224906193158153.json
convs/tweets_business_convs_30_1318971648079441922.json
convs/tweets_business_convs_30_1302325388539056129.json
convs/tweets_business_convs_30_1282191113890267138.json
convs/tweets_business_convs_30_1367556406996586501.json
convs/tweets_business_convs_30_1334563311141212164.json
convs/tweets_business_convs_30_1274053549874450434.json
convs/tweets_business_convs_30_1295399482692186112.json
convs/tweets_business_convs_30_1379894926444523522.json
convs/tweets_business_convs_30_1349498741359689728.json
convs/tweets_business_convs_30_1356733765544992769.json
convs/tweets_business_convs_30_13520184092534292

/tmp/ipykernel_185726/3140538378.py:1: DtypeWarning: Columns (38,58) have mixed types.Specify dtype option on import or set low_memory=False.
  for name, conv_id, df in get_conv_df(tweets_dir):


convs/tweets_police_100_convs_30_1396536689787609089.json
convs/tweets_police_100_convs_30_1396242374892851203.json
convs/tweets_police_100_convs_30_1396933248433803265.json
convs/tweets_police_100_convs_30_1396950151898533890.json
convs/tweets_police_100_convs_30_1396867303468412928.json
convs/tweets_black_ppl_convs_30_1379410705305239555.json
convs/tweets_black_ppl_convs_30_1384630952438214656.json
convs/tweets_black_ppl_convs_30_1376212170367758340.json
convs/tweets_black_ppl_convs_30_1377688742199181317.json
convs/tweets_black_ppl_convs_30_1378521085449211905.json
convs/tweets_black_ppl_convs_30_1387598276308918274.json
convs/tweets_black_ppl_convs_30_1385669809753317380.json
convs/tweets_black_ppl_convs_30_1381626511547985922.json
convs/tweets_black_ppl_convs_30_1394029987188019202.json
convs/tweets_black_ppl_convs_30_1383128428161994757.json
convs/tweets_black_ppl_convs_30_1390377000158724099.json
convs/tweets_black_ppl_convs_30_1384836410474180609.json
convs/tweets_black_ppl_con

In [63]:
for name, conv_id, df in get_conv_df(reddit_dir):
    g = make_reddit_graph(df)
    f = convs_dir / f"{name}_{conv_id}.json"
    json.dump(g, f.open('w'), indent=2)
    
    edges_csv = convs_dir / f"{name}_{conv_id}_edges.csv"
    out = csv.writer(edges_csv.open('w'))
    out.writerow(['source', 'target'])
    for edge in g['edges']:
        out.writerow(edge)
    
    nodes_csv = convs_dir / f"{name}_{conv_id}_nodes.csv"
    out = csv.DictWriter(nodes_csv.open('w'), fieldnames=['id', 'url', 'user', 'text', 'sentiment'])
    for node in g['nodes']:
        out.writerow(node)
        
    print(f)

convs/reddit_racial_wealth_gap_convs_30_m4ljb0.json
convs/reddit_racial_wealth_gap_convs_30_ifotnt.json
convs/reddit_racial_wealth_gap_convs_30_jwh0qy.json
convs/reddit_racial_wealth_gap_convs_30_kswcrt.json
convs/reddit_racial_wealth_gap_convs_30_kg1vjx.json
convs/reddit_racial_wealth_gap_convs_30_hbwbcl.json
convs/reddit_racial_wealth_gap_convs_30_ht2qiq.json
convs/reddit_racial_wealth_gap_convs_30_hggqkb.json
convs/reddit_racial_wealth_gap_convs_30_hwdozb.json
convs/reddit_racial_wealth_gap_convs_30_mz4jax.json
convs/reddit_racial_wealth_gap_convs_30_hwexm4.json
convs/reddit_racial_wealth_gap_convs_30_jq41ar.json
convs/reddit_racial_wealth_gap_convs_30_gvwp12.json
convs/reddit_racial_wealth_gap_convs_30_hibs7.json
convs/reddit_racial_wealth_gap_convs_30_mi4x7j.json
convs/reddit_racial_wealth_gap_convs_30_ioq56.json
convs/reddit_racial_wealth_gap_convs_30_n07vzw.json
convs/reddit_racial_wealth_gap_convs_30_ho9wg9.json
convs/reddit_racial_wealth_gap_convs_30_j84ala.json
convs/reddit_r